Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
from ultralytics import YOLO

from paddleocr import PaddleOCR
from pathlib import Path

ocr_en = PaddleOCR(use_angle_cls=True, lang='en')
ocr_ar = PaddleOCR(use_angle_cls=True, lang='ar')


license_plate_detection_model = YOLO('yolov11x-license-plate.pt')

car_detection_model = YOLO("yolov11n.pt")
    


/home/azoz-laptop/Desktop/Git-Projects/dlimageclass/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.
/tmp/ipykernel_683999/3619862382.py:6: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr_en = PaddleOCR(use_angle_cls=True, lang='en')
which: no ccache in (/home/azoz-laptop/Desktop/Git-Projects/dlimageclass/.venv/bin:/run/user/1000/fnm_multishells/105918_1767621208697/bin:/home/azoz-laptop/.local/share/fnm:/home/azoz-laptop/.turso:/home/azoz-laptop/.local/share/pnpm:/run/user/1000/fnm_multishells/2025_1767548146799/bin:/home/azoz-laptop/.local/share/fnm:/home/az

KeyboardInterrupt: 

In [ ]:
def detect_cars_and_plates(image_path: Path | str):
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path)
    output_plates = []
    for car in car_results[0].boxes:
        # Extract car bounding box
        x1, y1, x2, y2 = map(int, car.xyxy[0])
        car_image = car_results[0].orig_img[y1:y2, x1:x2]
        
        plate_results = license_plate_detection_model.predict(source=car_image)
        
        for plate in plate_results[0].boxes:
            # Extract license plate bounding box
            px1, py1, px2, py2 = map(int, plate.xyxy[0])
            license_plate_image = car_image[py1:py2, px1:px2]
            # ocr 
            ocr_result = ocr_en.predict(license_plate_image)
            print("Detected License Plate Text:", ocr_result)
            output_plates.append(ocr_result)
    return output_plates

In [ ]:
images = ['./bmwalleng.png']
for img_path in images:
    print(f"Processing image: {img_path}")
    plates = detect_cars_and_plates(img_path)
    print(f"Detected plates: {plates}")